# plotting quantified data

In [ ]:
import xarray as xr
from astropy import units as u

import astropy_xarray  # noqa: F401

# to be able to read unit attributes following the CF conventions
# import cf_xarray.units  # must be imported before pint_xarray
u.set_enabled_aliases(
    {
        "degK": u.Kelvin,
        "K": u.Kelvin,
        "degrees_north": u.deg,
        "degrees_east": u.deg,
    },
)

xr.set_options(display_expand_data=False)

## load the data

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature")
data = ds.air
data

## quantify the data

<div class="alert alert-info">
<strong>Note:</strong> this example uses the data provided by the <code>xarray.tutorial</code> functions. As such, the <code>units</code> attributes follow the CF conventions, which <code>pint</code> does not understand by default. To still be able to read them we are using the registry provided by <a href="https://github.com/calgray/cf-xarray"><tt>cf-xarray</tt></a>.
</div>

In [ ]:
quantified = data.astropy.quantify()
quantified

## work with the data

In [ ]:
monthly_means = (
    quantified.astropy.to("deg_C", equivalencies=u.temperature())
    .sel(time="2013")
    .groupby("time.month")
    .mean()
)
monthly_means

Most operations will preserve the units but there are some which will drop them (see the [duck array integration status](https://xarray.pydata.org/en/stable/user-guide/duckarrays.html#missing-features) page). To work around that there are unit-aware versions on the `.pint` accessor. For example, to select data use `.astropy.sel` instead of `.sel`:

In [ ]:
monthly_means.sel(
    lat=u.Quantity(4350, "arcmin"),
    lon=u.Quantity(12000, "arcmin"),
)

## plot

`xarray`'s plotting functions will cast the data to `numpy.ndarray`, so we need to "dequantify" first.

In [ ]:
monthly_means.astropy.dequantify(format="unicode").plot.imshow(col="month", col_wrap=4)